# Helper Function & Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
import pandas as pd
import numpy as np
import requests
import tftinsights
import tftinsights.config
import tftinsights.units

from tqdm.notebook import tqdm


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)

# --- Config ---
json_folder = "data"  # Folder where the json files are stored
output_csv = "csv/flattened_matches.csv" #to store json files as csv
traits_csv = "csv/traits_csv.csv" #trait will rate table
unit_csv = "csv/units_csv.csv"
current_set_prefix = tftinsights.config.CURRENT_SET_NAME
# --- Helper Function ---

    
def load_and_flatten_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
    # Example: Flattening `info.participants` from match data
    if 'info' in data and 'participants' in data['info']:   
        
        # Flatten the 'participants' section of the data
        df = pd.json_normalize(data['info']['participants'])
        df['match_id'] = data['metadata']['match_id']
        
        # Add other fields from 'info' (except 'participants') to the DataFrame
        for i in data['info']:
            if i != 'participants':
                df[i] = data['info'][i]

        return df
    else:
        print(f"Skipping {filepath}: unexpected format.")
        return pd.DataFrame()

def flatten_traits(traits):
    # Create {trait_name: tier} for each active trait (tier > 0)
    return {
        trait['name']: trait['tier_current']
        for trait in traits if trait.get('tier_current', 0) > 0
    }

def flatten_units(unit_list):
    result = {}
    for unit in unit_list:
        name = unit.get('character_id')
        tier = unit.get('tier', 0)
        #result[f"{name}_tier"] = tier  # Prevent key collision
        result[f"{name}"] = tier  # Prevent key collision
    return result

def flatten_items_count (unit_list):
    result = {}
    for unit in unit_list:
        items = unit.get ( 'itemNames', None )
        for i in items:
            if i is not None:
                key = f"item_{i}"
                if i in result:                          
                    result[key] += int(1)
                else:
                    result[key] = int(1)
    return result

                   
def flatten_unit_items(unit_list):
    result = {}
    for unit in unit_list:
        name = unit.get('character_id')
        items = unit.get('itemNames', 'None')
        padded = items + ['None'] * ( 3 - len ( items ))
        for i in range (3):
            result[f"{name}_item_{i}"] = padded[i]
    return result
    

def get_trait_info(traits):
    #print ( type ( traits) )
    # Create {trait_name: tier} for each active trait (tier > 0)
    return {
         f"{trait['name']}_{trait['tier_current']}": 
            {
                "tier_total": trait["tier_total"],
                "tier_current": trait["tier_current"]
            }
        for trait in traits if trait.get('tier_current', 0) > 0
    }
    
def reorder_and_clean_columns(combined_df):
    """
    Reorders and cleans the columns of the given DataFrame according to predefined rules.

    - Prioritized columns appear first.
    - Trait columns (starting with 'trait_') are sorted and next.
    - Remaining columns (not included elsewhere) follow, sorted alphabetically.
    - Unit columns (starting with 'unit_' but not containing '_item_') are next.
    - Item columns (containing '_item_') are last.

    Fills missing values in:
    - trait and unit columns with 0 and converts them to int.
    - item columns with 'None' and converts them to str.
    """
    # Prioritized columns
    prioritized_columns = ['riotIdGameName', 'riotIdTagline',
                           'match_id', 'placement', 'win', 'level',
                           'total_damage_to_players', 'players_eliminated']
    
    # Identify columns
    traits_columns = sorted([col for col in combined_df.columns if col.startswith('trait_')])

    units_columns = sorted([
        col for col in combined_df.columns 
        if col.startswith('unit_') and '_item_' not in col
    ])

    item_count_columns = sorted([
        col for col in combined_df.columns 
        if col.startswith('item_')
    ])

    # print ("ITEM COUNT COLUMNS")
    # print ( list ( item_count_columns ) )
    item_columns = sorted([
        col for col in combined_df.columns 
        if '_item_' in col
    ])
   
    # Remaining columns
    already_included = set(prioritized_columns + traits_columns + units_columns + item_columns + item_count_columns)
    remaining_columns = sorted([col for col in combined_df.columns if col not in already_included])
    
    # Final column order
    final_column_order = prioritized_columns + traits_columns + remaining_columns + units_columns + item_columns + item_count_columns
    combined_df = combined_df[final_column_order]
    
    # Data type conversions
    combined_df[item_count_columns] = combined_df[item_count_columns].fillna(0).astype(int)
    combined_df[traits_columns]     = combined_df[traits_columns    ].fillna(0).astype(int)
    combined_df[units_columns]      = combined_df[units_columns     ].fillna(0).astype(int)
    combined_df[item_columns]       = combined_df[item_columns      ].fillna('None').astype(str)    
    
    return combined_df
    
def rebuild_csv(df2, output_csv, write_header=True):
    """Rebuild the CSV with the new data and schema"""
    print("🔁 Rebuilding CSV due to new columns...")


    if os.path.exists(output_csv):
        df1 = pd.read_csv(output_csv, low_memory=False)

        all_columns = sorted(set(df1.columns).union(df2.columns))  # ['a', 'b', 'c', 'd']
       
        df1 = df1.reindex(columns=all_columns)
        df2 = df2.reindex(columns=all_columns)   

        df1.reset_index(drop=True, inplace=True)
        df2.reset_index(drop=True, inplace=True)
        combined_df = pd.concat([df1, df2], ignore_index=True, sort=False)   
    else:
        # Ensure consistent column order in the new CSV
        combined_df = df2

    
    combined_df = reorder_and_clean_columns ( combined_df )
    combined_df.to_csv(output_csv, index=False, mode='w', header=True)

    return combined_df


In [4]:
import os
import pandas as pd
from tqdm import tqdm

"""
Processes TFT match JSON files into a flattened CSV. Appends only unprocessed matches.
Trait aggregation must be handled separately.
"""
if os.path.exists(output_csv):
    os.remove(output_csv)

#Track seen match_ids if output exists

full_column_set = set()
seen_match_ids = set()
write_header = True
match_add_count = 0 

if os.path.exists(output_csv):
    processed_df = pd.read_csv(output_csv)
    seen_match_ids = set(processed_df["match_id"])
    fileheaders = pd.read_csv(output_csv, nrows=0).columns.tolist()
    full_column_set = set ( fileheaders )
    write_header = False

json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]
print(f"📁 {len(json_files)} JSON files found in folder")

for filename in tqdm(json_files, desc="Processing JSONs"):
    filepath = os.path.join(json_folder, filename)

    # if match_add_count > 1000:
    #     break
    
    df = load_and_flatten_json(filepath)

    if df.empty or "match_id" not in df.columns:
        print ( " skip ")
        continue

    match_id = df["match_id"].iloc[0]
    if match_id in seen_match_ids:
        continue  # Skip already processed

    # Sort placements for consistency
    df.sort_values(by='placement', inplace=True)
   
    # Defensive flattening for traits
    flattened_traits = df['traits'].apply(flatten_traits)
    all_trait_keys = set().union(*flattened_traits)
    traits_flat = flattened_traits.apply(
        lambda d: {k: d.get(k, 0) for k in all_trait_keys}
        ).apply(pd.Series).astype(int)
    traits_flat.columns = [f"trait_{c}" for c in traits_flat.columns]

    # Defensive flattening for units        
    flattened_units = df['units'].apply(flatten_units)
    all_unit_keys = set().union(*flattened_units)
    units_flat = flattened_units.apply(
        lambda d: {k: d.get(k, 0) for k in all_unit_keys}
        ).apply(pd.Series)
    units_flat.columns = [f"unit_{c}" for c in units_flat.columns]
    units_flat = units_flat.fillna(0).astype(int)

    # Flatten items - by unit
    flattened_items = df['units'].apply(flatten_unit_items)        
    item_keys = set().union(*flattened_items)
    items_flat = flattened_items.apply(pd.Series)
    items_flat = items_flat.fillna('None').astype(str)

    # Flatten items #2 - by count
    flattened_item_2 = df['units'].apply(flatten_items_count)        
    item_keys_2 = set().union(*flattened_item_2)
    items_flat2 = flattened_item_2.apply(pd.Series)
    items_flat2 = items_flat2.fillna(0).astype(int)
    
    
    # Combine everything
    df = pd.concat([
        df.drop(columns=[ 'units']),
        #traits_flat,
        #
        items_flat,
        units_flat,
        traits_flat,
        items_flat2
    ], axis=1)

    match_add_count = match_add_count + 1
    seen_match_ids.add(match_id)

    #print ( f"match_add_count = { match_add_count } " )
    # Track full column set and check for new columns
    new_cols = set(df.columns)
    if not new_cols.issubset(full_column_set):
        
        new_columns = new_cols - full_column_set        
        if len (new_columns) < 5:
            print("New columns detected:", new_columns)
        else:
            print("New columns detected:", len (new_columns))

        fileheaders = rebuild_csv(df, output_csv).columns       
        full_column_set.update ( fileheaders )
        write_header = False            
    else:
        #print ( "----------NO NEW COLUMM-------")
        
        df = df.reindex(columns=fileheaders)
        df = reorder_and_clean_columns ( df )        
        df.to_csv(output_csv, mode='a', header=False, index=False)
        write_header = False


    # if match_add_count >= 1000:
    #     break
    
    # except Exception as e:
    #     print(f"❌ Error processing {filename}: {e}")

print(f"✅ Completed processing. Appended new matches to {output_csv}.")


📁 35027 JSON files found in folder


Processing JSONs:   0%|                                                              | 3/35027 [00:00<27:55, 20.90it/s]

New columns detected: 278
🔁 Rebuilding CSV due to new columns...
New columns detected: 58
🔁 Rebuilding CSV due to new columns...
New columns detected: 15
🔁 Rebuilding CSV due to new columns...
New columns detected: 5
🔁 Rebuilding CSV due to new columns...


Processing JSONs:   0%|                                                              | 6/35027 [00:00<31:17, 18.65it/s]

New columns detected: 7
🔁 Rebuilding CSV due to new columns...
New columns detected: 26
🔁 Rebuilding CSV due to new columns...
New columns detected: {'item_TFT_Item_Artifact_LightshieldCrest', 'item_TFT_Item_Artifact_WitsEnd', 'item_TFT14_JhinCyberneticItem', 'item_TFT4_Item_OrnnMuramana'}
🔁 Rebuilding CSV due to new columns...


...

New columns detected: 7
🔁 Rebuilding CSV due to new columns...


Processing JSONs:  28%|████████████████▍                                          | 9762/35027 [08:55<26:36, 15.83it/s]

New columns detected: {'TFT14_AnnieTibbers_item_0', 'TFT14_AnnieTibbers_item_2', 'unit_TFT14_AnnieTibbers', 'TFT14_AnnieTibbers_item_1'}
🔁 Rebuilding CSV due to new columns...


Processing JSONs: 100%|██████████████████████████████████████████████████████████| 35027/35027 [31:28<00:00, 18.54it/s]

✅ Completed processing. Appended new matches to csv/flattened_matches.csv.


# MAIN - Transform data into DF
This method was acceptable in the start when the data set is small, but as it has grown we started hitting technical performane issues. 

In order to fix this, we'll need to append to the CSV instead of keeping the whole thing as a DF.
This will take some refractor time.

In [3]:
from tftinsights.units import get_mapping_table_for_traits
import pandas as pd
import time
"""
Rebuilds the trait_set by reading the flattened match CSV and extracting traits.
Saves result to `traits_csv`.
"""

trait_set = pd.DataFrame()
df = pd.read_csv("csv/flattened_matches.csv", low_memory=False)

for traits in tqdm(df['traits'], desc="Processing traits"):
    traits = traits.replace ( "'","\"")
    traits_json = json.loads( traits )
    trait_df = pd.DataFrame(traits_json)    
    if not trait_df.empty:
        trait_set  = pd.concat([trait_set, trait_df], axis=0, ignore_index=True)
        trait_set = trait_set[trait_set['tier_current'] != 0]
        trait_set.drop_duplicates(inplace=True)
        trait_set = trait_set.loc[trait_set.groupby(['name', 'tier_current'])['num_units'].idxmin()]    

print(f"✅ Rebuilt trait set and saved to {traits_csv}")

trait_set = trait_set.rename(columns={'name': 'id'})
trait_names = get_mapping_table_for_traits()
merged_df = pd.merge(trait_names, trait_set, on='id', how='inner')
merged_df.to_csv('csv/traits_csv.csv', index=False)

#TODO - Build an Early out, when no new rows have been added for an extended period

Processing traits: 100%|██████████████████████████████████████████████████████| 216159/216159 [04:42<00:00, 765.40it/s]


✅ Rebuilt trait set and saved to csv/traits_csv.csv


In [85]:
merged_df

,id,name,num_units,style,tier_current,tier_total
0,TFT14_Immortal,Golden Ox,2,1,1,3
1,TFT14_Immortal,Golden Ox,4,2,2,3
2,TFT14_Immortal,Golden Ox,6,4,3,3
3,TFT14_Cutter,Executioner,2,1,1,4
4,TFT14_Cutter,Executioner,3,2,2,4
5,TFT14_Cutter,Executioner,4,2,3,4
6,TFT14_Cutter,Executioner,5,4,4,4
7,TFT14_Strong,Slayer,2,1,1,3
8,TFT14_Strong,Slayer,4,2,2,3
9,TFT14_Strong,Slayer,6,4,3,3


In [82]:
from tftinsights.units import get_mapping_table_for_traits

traits_level_data = pd.read_csv('csv/traits_raw_csv.csv')
id_to_name_mapping = get_mapping_table_for_traits()
traits_level_data = traits_level_data.merge(id_to_name_mapping, on='name', how='inner')
traits_level_data = traits_level_data.reset_index(drop=True) #for some reason the index was saving this to the csv 
traits_level_data.to_csv('csv/traits_csv.csv',index=False)